# Kickstarter dataset
## Prédire la probabilité de valider une campagne de financement participatif en travaillant le type de projet, le montant des fonds demandés et la durée de la campagne. 


In [65]:
import seaborn as sns
import pandas as pd
import numpy as np

dt = pd.read_csv("/content/drive/My Drive/Colab Notebooks/ks-projects-201801.csv")
dt

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378656,999976400,ChknTruk Nationwide Charity Drive 2014 (Canceled),Documentary,Film & Video,USD,2014-10-17,50000.0,2014-09-17 02:35:30,25.0,canceled,1,US,25.0,25.0,50000.00
378657,999977640,The Tribe,Narrative Film,Film & Video,USD,2011-07-19,1500.0,2011-06-22 03:35:14,155.0,failed,5,US,155.0,155.0,1500.00
378658,999986353,Walls of Remedy- New lesbian Romantic Comedy f...,Narrative Film,Film & Video,USD,2010-08-16,15000.0,2010-07-01 19:40:30,20.0,failed,1,US,20.0,20.0,15000.00
378659,999987933,BioDefense Education Kit,Technology,Technology,USD,2016-02-13,15000.0,2016-01-13 18:13:53,200.0,failed,6,US,200.0,200.0,15000.00


preprocessing

In [66]:
# Isoler X

# Drop data non pertinente
X = dt.drop(['name','ID','currency','pledged','state','backers','country','usd pledged','usd_pledged_real','usd_goal_real','deadline','launched'], axis='columns') 

X['category'] = X['category'].astype('category').cat.codes
X['main_category'] = X['main_category'].astype('category').cat.codes

In [67]:
#calculer dates en nombres de jours
import time as timelibrary
import datetime

def get_delta(end,start):
  list_duration = []
  for i in range(len(end)):
    date, time = start[i].split(" ")
    end_date = timelibrary.mktime(datetime.datetime.strptime(end[i], "%Y-%m-%d").timetuple())
    start_date = timelibrary.mktime(datetime.datetime.strptime(date, "%Y-%m-%d").timetuple())
    list_duration.append(end_date - start_date)
  return list_duration

X['date_duration'] = get_delta(dt['deadline'],dt['launched'])

In [68]:
# Isoler Y (target) 
Y = dt['state'].str.contains('canceled').replace(False, 'failed') # Interpréter annulé comme un échec
Y = Y.astype('category').cat.codes
Y

0         1
1         1
2         1
3         1
4         0
         ..
378656    0
378657    1
378658    1
378659    1
378660    1
Length: 378661, dtype: int8

In [69]:
from sklearn.model_selection import train_test_split

X_tr, X_te, Y_tr, Y_te = train_test_split(X, Y, random_state=15, stratify=Y)

In [70]:
from sklearn.neighbors import KNeighborsClassifier as KNN

knn = KNN(n_neighbors=3)
knn.fit(X_tr, Y_tr)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [71]:
def accuracy(preds, target):
    M = target.shape[0] # Nombre de lignes
    total_correctes = (preds == target).sum()
    return total_correctes / M # Accuracy

predictions = knn.predict(X_te)
accuracy(predictions, Y_te)

0.8745906661314516